In [11]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from collections import defaultdict

%load_ext autoreload
%autoreload 2


from utils import get_percentage_of_image, normalize_image
import torch.nn.functional as F
import torch


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
MODEL_TYPE = "convnext_tiny_test_NOSD"
XAI_TYPE = "gradcam"
BASE_DIR = "data/"
DATASET = "atsds_large"
DATASET_SPLIT = "test"
GROUND_TRUTH_DIR = BASE_DIR + DATASET + "_mask/" + DATASET_SPLIT
BACKGROUND_DIR = BASE_DIR + DATASET + "_background/" + DATASET_SPLIT
DATASET_DIR = BASE_DIR + DATASET + "/" + DATASET_SPLIT
XAI_DIR = BASE_DIR + "auswertung/" + MODEL_TYPE + "/" + XAI_TYPE + "/" + DATASET_SPLIT + "/"
ADV_FOLDER = BASE_DIR + "auswertung/" + MODEL_TYPE + "/" + XAI_TYPE + "/adversarial/"
print(XAI_DIR)

data/auswertung/convnext_tiny_test_NOSD/gradcam/test/


In [13]:
IMAGES_PATH = 'data/' + DATASET + '/' + DATASET_SPLIT + '/'
# Define our Categories
CATEGORIES = sorted(os.listdir(IMAGES_PATH))
print(CATEGORIES)
class_to_dataset_class_dict = {}
for cat in CATEGORIES:
    class_to_dataset_class_dict[cat] = cat
label_idx_dict = {}
for count,cat in enumerate(CATEGORIES):
    label_idx_dict[cat] = count

imagedict = {}
for cat in CATEGORIES:
    imagedict[cat] = []
    imagelist = os.listdir(IMAGES_PATH + cat + "/")
    for im in imagelist:
        imagedict[cat].append(im)            

['00001', '00002', '00003', '00004', '00005', '00007', '00008', '00009', '00010', '00011', '00012', '00013', '00014', '00017', '00018', '00025', '00031', '00035', '00038']


In [14]:
avg_pooling =torch.nn.AvgPool2d((129,129),stride = 1, padding=64,count_include_pad=False)

In [15]:
for cat in CATEGORIES:
    images = imagedict[cat]
    if not os.path.isdir(XAI_DIR + cat + "/grad_mask/"):
        os.makedirs(XAI_DIR + cat + "/grad_mask/" )
    for imagename in images:
        mask_raw = np.load(XAI_DIR + cat + "/mask/" + imagename + ".npy")
        mask = torch.Tensor(np.array(mask_raw)).unsqueeze(0).unsqueeze(0)
        pooled_mask = avg_pooling(mask)
        grad_mask = (normalize_image(np.array(mask_raw)) + normalize_image(pooled_mask.squeeze().squeeze().numpy())/100)
        np.save(XAI_DIR + cat + "/grad_mask/" + imagename,grad_mask)
        #save_mask = Image.fromarray(grad_mask*255).convert("L")
        #save_mask.save(XAI_DIR + cat + "/grad_mask/" + imagename)
        #np.save(grad_mask)        

In [10]:
### MULTI 
XAI_TYPES_MULTI = ["lime","gradcam","prism"]
for current_xai in XAI_TYPES_MULTI:
    XAI_DIR = BASE_DIR + "auswertung/" + MODEL_TYPE + "/" + current_xai + "/" + DATASET_SPLIT + "/"
    for cat in CATEGORIES:
        images = imagedict[cat]
        if not os.path.isdir(XAI_DIR + cat + "/grad_mask/"):
            os.makedirs(XAI_DIR + cat + "/grad_mask/" )
        for imagename in images:
            mask_raw = np.load(XAI_DIR + cat + "/mask/" + imagename + ".npy")
            mask = torch.Tensor(np.array(mask_raw)).unsqueeze(0).unsqueeze(0)
            pooled_mask = avg_pooling(mask)
            grad_mask = (normalize_image(np.array(mask_raw)) + normalize_image(pooled_mask.squeeze().squeeze().numpy())/100)
            np.save(XAI_DIR + cat + "/grad_mask/" + imagename,grad_mask)
            #save_mask = Image.fromarray(grad_mask*255).convert("L")
            #save_mask.save(XAI_DIR + cat + "/grad_mask/" + imagename)
            #np.save(grad_mask)        